In [2]:

def languageClassifier(filename, doc2vec):
    
    #loading data
    data = pd.read_csv(filename)
    d2v = pd.read_csv(doc2vec, index_col = False, delimiter = ',', header = None)
    #print(d2v)
    
    
    #combining doc2vec and language into a df
    languages = data["Language"]
    #print(d2v.shape)
   # print(languages)
    d2v["Language"] = languages
    #print(d2v)
    count = 0

    
    #sorting values by language, pruning NaNs
    knownLanguage = d2v.dropna()
    languages = languages.dropna()
    
    
    #sorting remaining instances by language
    data_by_lang = [(x,y.iloc[:,:]) for x, y in knownLanguage.groupby(languages)]
    
    #calculating centroids of each language
    langCentroids = []
    for language, df in data_by_lang:
        df.pop("Language")
        df.loc[language] = df.mean()
        df = df.iloc[-1]
        
        langCentroids.append(df)
        
    
    i = 0
    
   
    while i < len(d2v.index):
        
        if pd.isna(d2v.at[i, "Language"]):
            
            currRow = d2v.iloc[i].iloc[:-1]
            
            bestLang = langClass(langCentroids, currRow)
            d2v.at[i, "Language"] = bestLang
        i += 1
    
    
    
    bestLang = langClass(langCentroids, currRow)
    #print(currRow)
   # print(langCentroids)
    print(bestLang)
    print(d2v)
   
    return d2v

doc2vecLangs = languageClassifier("Data/book_rating_train.csv", "Data/book_text_features_doc2vec/train_desc_doc2vec100.csv")

NameError: name 'langClass' is not defined

In [ ]:
#################distance and similarity measures used ################

def cosine_sim(v1, v2):
    #note v1 and v2 are pandas series of equal length
    
    i = 0
    dotProduct = 0
    absV1 = 0
    absV2 = 0
    
    while i < len(v1):
        dotProduct += v1[i]*v2[i]
        absV1 += v1[i]**2
        absV2 += v2[i]**2
        
        i+=1
        
    absV1 = np.sqrt(absV1)
    absV2 = np.sqrt(absV2)
    return dotProduct / (absV1 * absV2)

def manDistance(v1, v2):
    
    i = 0
    totalDistance = 0
    while i < len(v1):
        totalDistance = np.abs(v1[i] - v2[i])
        i += 1
        
    return totalDistance

In [12]:
# functions for finding the best language class, first is more robust

def langClass(centroids, row, mode = "cosine"):
    
    simScores = {}
    print(row.name)
    for centroid in centroids:
        if mode == "cosine":
            simScores[centroid.name] = cosine_sim(centroid, row)
        else:
            simScores[centroid.name] = manDistance(centroid, row)
        
    #setting eng as base lang
    maxSim = simScores["fre"]
    bestLang = "fre"
    for lang, sim in simScores.items():
        
        #print(f"lang is {lang}, sim is {sim}")
        
        if mode == "cosine":
            if sim > maxSim:
                maxSim = sim
                bestLang = lang
                
        else:
            if sim < maxSim:
                maxSim = sim
                bestLang = lang
    return bestLang


def langClass2(centroids, row):
    
    simScores = {}
    print(row.name)
    for centroid in centroids:
         simScores[centroid.name] = manDistance(centroid, row)
        
    #setting eng as base lang
    maxSim = simScores["fre"]
    bestLang = "fre"
    for lang, sim in simScores.items():
        
        #print(f"lang is {lang}, sim is {sim}")
    
        if sim < maxSim:
            maxSim = sim
            bestLang = lang
            
    return bestLang


In [28]:
print("Language Counts with Cosine Similarity")
print(manDistClass["Language"].value_counts())

print("\nLanguage Counts with Manhattan Distance")

print(manDist2Class["Language"].value_counts())


Language Counts with Cosine Similarity
eng    14692
lat     1612
zho     1516
frs     1028
spa      732
jpn      569
mul      544
fre      467
ger      381
per      350
ita      333
grc      271
rus      169
por      135
swe      105
heb       99
nld       32
ara       28
Name: Language, dtype: int64

Language Counts with Manhattan Distance
eng    6673
frs    2947
swe    2880
grc    2002
ger    1365
ita    1150
ara    1028
por     908
spa     692
nld     630
heb     625
mul     617
rus     602
jpn     359
fre     315
zho     128
per      87
lat      55
Name: Language, dtype: int64


In [16]:
'''Same as previous one, but using Manhattan distance, very slow,least reliable'''

def languageClassifier2(filename, doc2vec):
    
    #loading data
    data = pd.read_csv(filename)
    d2v = pd.read_csv(doc2vec, index_col = False, delimiter = ',', header = None)
    #print(d2v)
    
    
    #combining doc2vec and language into a df
    languages = data["Language"]
    #print(d2v.shape)
   # print(languages)
    d2v["Language"] = languages
    #print(d2v)
    count = 0

    
    #sorting values by language, pruning NaNs
    knownLanguage = d2v.dropna()
    languages = languages.dropna()
    
    
    #sorting remaining instances by language
    data_by_lang = [(x,y.iloc[:,:]) for x, y in knownLanguage.groupby(languages)]
    
    #calculating centroids of each language
    langCentroids = []
    for language, df in data_by_lang:
        df.pop("Language")
        df.loc[language] = df.mean()
        df = df.iloc[-1]
        
        langCentroids.append(df)
        
    
    i = 0
    
   
    while i < len(d2v.index):
        
        if pd.isna(d2v.at[i, "Language"]):
            
            currRow = d2v.iloc[i].iloc[:-1]
            
            bestLang = langClass2(langCentroids, currRow)
            d2v.at[i, "Language"] = bestLang
        i += 1
    
    
    
    
    #print(currRow)
   # print(langCentroids)
    print(bestLang)
    print(d2v)
   
    return d2v


manDist2Class = languageClassifier2("Data/book_rating_train.csv", "Data/book_text_features_doc2vec/train_desc_doc2vec100.csv")

0
1
2
3
5
6
7
9
10
11
12
13
14
15
17
18
19
20
22
25
27
28
30
31
32
33
35
36
37
38
40
41
42
44
46
47
49
50
51
52
53
54
55
57
59
60
61
62
64
65
66
67
69
70
71
72
73
75
76
79
81
83
84
85
87
88
89
91
94
95
97
98
99
100
101
102
105
108
109
112
114
115
117
118
119
120
121
122
123
124
125
126
127
129
130
131
132
133
135
136
137
139
142
145
146
148
149
151
152
153
154
155
159
160
162
163
164
166
167
168
169
170
171
173
174
175
177
179
180
181
182
183
184
185
186
187
188
189
194
195
196
197
200
202
203
208
209
212
213
215
217
218
221
222
223
224
226
227
230
231
232
233
234
236
238
240
241
242
244
245
246
247
249
250
251
253
255
257
259
260
261
262
263
266
268
269
270
271
272
273
274
275
277
278
280
281
283
285
287
288
289
290
291
292
294
297
299
300
305
307
308
309
310
311
312
313
314
317
318
319
321
323
324
326
327
328
329
332
333
334
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
353
354
355
356
360
361
363
364
366
367
368
369
370
371
372
375
376
377
378
379
380
381
382
388
389
3

2434
2435
2436
2437
2438
2439
2441
2442
2443
2444
2446
2447
2449
2450
2451
2453
2454
2455
2456
2458
2459
2461
2463
2464
2465
2466
2467
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2485
2486
2488
2489
2490
2492
2494
2495
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2511
2513
2514
2515
2519
2520
2522
2523
2524
2526
2527
2528
2529
2530
2531
2532
2533
2534
2536
2538
2539
2540
2541
2542
2543
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2560
2561
2562
2563
2564
2565
2566
2567
2568
2569
2570
2571
2572
2573
2574
2577
2578
2579
2581
2582
2583
2584
2586
2588
2589
2590
2591
2592
2593
2594
2595
2596
2597
2598
2599
2600
2601
2602
2603
2605
2606
2607
2608
2610
2611
2612
2613
2614
2615
2616
2617
2618
2623
2625
2626
2627
2628
2630
2631
2632
2635
2637
2638
2639
2640
2641
2644
2646
2647
2648
2650
2651
2656
2657
2658
2659
2661
2664
2666
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2681
2682
2683
2684
2685
2686
2687
2688
2690
2693
2694
2695
2696


4622
4623
4624
4625
4626
4627
4628
4629
4630
4631
4633
4634
4635
4636
4637
4638
4639
4643
4645
4648
4650
4651
4652
4654
4655
4656
4658
4661
4662
4663
4664
4665
4666
4667
4668
4669
4670
4672
4673
4676
4677
4678
4680
4681
4683
4684
4685
4686
4687
4688
4689
4690
4691
4692
4695
4698
4700
4702
4703
4705
4707
4708
4709
4714
4715
4716
4718
4719
4721
4723
4724
4727
4728
4729
4733
4737
4738
4741
4742
4743
4744
4745
4746
4747
4750
4753
4754
4755
4756
4758
4759
4761
4763
4765
4766
4767
4768
4769
4770
4772
4773
4774
4776
4777
4778
4779
4781
4782
4783
4785
4787
4788
4789
4790
4791
4792
4793
4795
4796
4797
4799
4800
4801
4802
4803
4804
4805
4806
4808
4809
4810
4812
4813
4814
4815
4816
4819
4820
4821
4823
4824
4826
4827
4828
4829
4830
4831
4832
4833
4834
4839
4840
4841
4842
4844
4845
4846
4848
4849
4850
4851
4852
4853
4854
4855
4857
4858
4860
4861
4862
4866
4868
4869
4870
4872
4873
4874
4875
4880
4881
4882
4883
4884
4886
4887
4889
4890
4891
4892
4894
4895
4896
4897
4899
4901
4902
4903
4904
4905
4906


6855
6856
6857
6858
6859
6860
6861
6863
6864
6868
6871
6872
6874
6875
6877
6878
6879
6880
6881
6882
6883
6885
6886
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6900
6901
6902
6903
6904
6907
6908
6910
6911
6912
6913
6914
6915
6916
6918
6919
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6932
6933
6934
6936
6937
6939
6940
6942
6943
6945
6946
6947
6948
6949
6951
6953
6954
6955
6956
6958
6960
6961
6963
6964
6965
6967
6968
6969
6970
6972
6973
6977
6978
6979
6981
6982
6983
6985
6986
6990
6991
6992
6993
6999
7002
7003
7004
7005
7006
7007
7009
7010
7015
7017
7018
7020
7026
7027
7028
7029
7030
7032
7033
7034
7035
7036
7037
7039
7040
7041
7044
7046
7048
7050
7052
7054
7055
7056
7057
7059
7060
7061
7062
7063
7064
7067
7068
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7093
7094
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7109
7111
7114
7115
7116
7117
7119
7121
7122
7123
7124
7125
7127
7128
7129
7131
7134
7136
7137


9083
9086
9087
9088
9089
9090
9092
9094
9095
9096
9097
9098
9099
9101
9102
9103
9104
9105
9106
9107
9108
9110
9111
9112
9113
9114
9115
9116
9120
9121
9122
9123
9125
9126
9127
9128
9129
9130
9131
9132
9133
9134
9136
9137
9138
9139
9140
9141
9143
9145
9147
9148
9149
9150
9151
9152
9153
9155
9156
9157
9158
9160
9161
9163
9165
9166
9168
9169
9170
9171
9173
9174
9175
9176
9178
9179
9181
9182
9183
9185
9186
9187
9189
9190
9191
9193
9194
9196
9197
9198
9201
9202
9203
9205
9206
9207
9208
9209
9210
9211
9212
9213
9215
9216
9217
9218
9219
9220
9222
9223
9224
9226
9227
9231
9232
9233
9235
9236
9237
9238
9239
9240
9241
9243
9244
9246
9247
9249
9250
9251
9256
9257
9258
9259
9260
9261
9262
9263
9265
9266
9267
9269
9270
9271
9273
9274
9275
9276
9277
9279
9282
9283
9284
9285
9286
9288
9289
9290
9291
9292
9293
9294
9296
9298
9299
9300
9301
9302
9303
9304
9305
9307
9308
9309
9310
9311
9312
9313
9314
9315
9316
9317
9319
9320
9321
9322
9323
9324
9325
9326
9327
9328
9329
9332
9333
9334
9335
9336
9337
9338


...

14750
14752
14753
14754
14755
14757
14758
14760
14762
14764
14766
14767
14768
14769
14771
14773
14774
14775
14776
14777
14778
14779
14781
14782
14783
14784
14785
14786
14787
14788
14790
14792
14793
14794
14798
14800
14801
14802
14803
14805
14806
14807
14808
14810
14811
14814
14815
14817
14818
14820
14822
14823
14824
14825
14826
14827
14830
14831
14832
14833
14834
14835
14836
14837
14838
14839
14840
14842
14843
14844
14845
14846
14847
14848
14851
14852
14853
14855
14856
14857
14858
14859
14860
14861
14862
14863
14865
14866
14869
14871
14872
14873
14874
14875
14876
14878
14882
14883
14884
14886
14887
14888
14889
14891
14892
14893
14895
14896
14897
14899
14901
14902
14903
14904
14906
14907
14908
14909
14912
14913
14914
14915
14916
14917
14919
14921
14922
14923
14924
14925
14927
14928
14929
14930
14931
14932
14933
14934
14935
14936
14937
14938
14939
14940
14942
14943
14945
14947
14948
14949
14951
14952
14953
14954
14957
14958
14962
14963
14964
14965
14966
14967
14969
14970
14971
14972
1497

16619
16620
16621
16622
16623
16626
16627
16628
16629
16631
16632
16634
16635
16636
16637
16638
16640
16641
16642
16644
16645
16646
16647
16648
16649
16650
16651
16653
16656
16657
16658
16659
16660
16662
16663
16664
16665
16666
16667
16668
16669
16670
16671
16672
16674
16675
16677
16678
16679
16680
16683
16684
16685
16686
16687
16688
16690
16691
16692
16693
16694
16697
16698
16699
16700
16701
16702
16703
16704
16706
16707
16708
16710
16711
16712
16713
16714
16715
16716
16717
16718
16720
16721
16722
16723
16726
16727
16728
16729
16730
16731
16735
16736
16737
16738
16739
16740
16742
16743
16744
16746
16748
16749
16750
16751
16752
16754
16755
16757
16758
16760
16761
16763
16764
16766
16767
16769
16770
16771
16772
16773
16775
16776
16778
16779
16780
16783
16786
16787
16789
16791
16793
16794
16796
16798
16801
16802
16803
16804
16805
16808
16809
16811
16812
16813
16815
16816
16818
16819
16821
16822
16824
16825
16827
16828
16829
16830
16831
16832
16833
16834
16836
16837
16838
16839
16840
1684

18411
18412
18416
18418
18419
18420
18422
18424
18425
18426
18427
18430
18431
18432
18433
18434
18436
18437
18438
18440
18442
18443
18444
18445
18446
18447
18448
18450
18452
18453
18454
18455
18456
18459
18461
18462
18463
18464
18465
18466
18467
18470
18471
18473
18474
18475
18476
18477
18479
18480
18481
18482
18483
18484
18485
18486
18488
18489
18490
18491
18492
18494
18495
18496
18497
18499
18500
18501
18502
18503
18504
18505
18506
18507
18508
18510
18511
18512
18513
18514
18516
18517
18518
18519
18520
18521
18522
18523
18524
18526
18527
18528
18529
18530
18531
18532
18533
18534
18535
18536
18537
18538
18539
18540
18541
18543
18544
18545
18547
18549
18550
18551
18554
18555
18556
18557
18560
18561
18562
18563
18564
18565
18566
18567
18570
18571
18572
18573
18574
18575
18576
18577
18578
18579
18580
18581
18584
18585
18587
18588
18589
18591
18592
18594
18596
18599
18600
18601
18602
18603
18605
18607
18609
18612
18614
18615
18616
18617
18618
18620
18622
18623
18624
18625
18626
18627
1862

20261
20262
20263
20264
20265
20266
20268
20269
20270
20271
20272
20273
20274
20275
20276
20277
20280
20281
20284
20288
20289
20292
20293
20294
20295
20297
20298
20299
20300
20301
20302
20303
20304
20305
20306
20307
20308
20309
20310
20311
20312
20313
20314
20315
20316
20317
20318
20319
20321
20322
20323
20324
20325
20326
20327
20328
20330
20331
20332
20333
20335
20338
20339
20340
20341
20342
20345
20346
20347
20348
20349
20350
20351
20352
20353
20355
20356
20357
20360
20361
20362
20363
20364
20366
20367
20368
20369
20370
20372
20373
20374
20375
20376
20378
20379
20380
20381
20382
20383
20384
20385
20386
20387
20388
20389
20390
20391
20392
20393
20394
20395
20397
20399
20402
20403
20405
20406
20407
20409
20411
20412
20413
20415
20416
20417
20418
20419
20421
20422
20423
20424
20425
20426
20428
20429
20430
20431
20433
20434
20435
20436
20437
20438
20440
20442
20443
20444
20445
20446
20447
20448
20449
20450
20451
20452
20453
20456
20458
20459
20461
20462
20463
20464
20466
20469
20470
2047

22074
22075
22076
22077
22079
22080
22081
22083
22084
22085
22086
22088
22090
22091
22092
22093
22094
22096
22097
22098
22100
22103
22104
22105
22106
22107
22108
22109
22110
22111
22114
22115
22116
22117
22118
22119
22120
22122
22123
22124
22125
22126
22127
22128
22129
22130
22131
22132
22133
22134
22135
22137
22139
22140
22141
22142
22144
22147
22149
22150
22152
22154
22155
22157
22158
22161
22163
22164
22165
22166
22167
22168
22169
22170
22171
22173
22174
22175
22177
22178
22179
22180
22181
22182
22183
22186
22187
22188
22191
22192
22193
22194
22195
22196
22197
22198
22199
22200
22201
22204
22205
22206
22207
22208
22209
22210
22211
22212
22213
22214
22215
22216
22219
22220
22221
22222
22223
22224
22225
22226
22227
22230
22234
22235
22236
22238
22239
22240
22242
22243
22244
22246
22247
22248
22249
22250
22251
22253
22254
22255
22256
22257
22258
22262
22263
22264
22266
22267
22268
22270
22273
22274
22275
22276
22277
22278
22279
22280
22283
22285
22286
22287
22288
22289
22290
22292
2229

In [ ]:
'''Uses Cosine similarity as metric, bad, takes too long, seems not good'''
def languageClassifier(filename, doc2vec):
    
    #loading data
    data = pd.read_csv(filename)
    d2v = pd.read_csv(doc2vec, index_col = False, delimiter = ',', header = None)
    #print(d2v)
    
    
    #combining doc2vec and language into a df
    languages = data["Language"]
    #print(d2v.shape)
   # print(languages)
    d2v["Language"] = languages
    #print(d2v)
    count = 0

    
    #sorting values by language, pruning NaNs
    knownLanguage = d2v.dropna()
    languages = languages.dropna()
    
    
    #sorting remaining instances by language
    data_by_lang = [(x,y.iloc[:,:]) for x, y in knownLanguage.groupby(languages)]
    
    #calculating centroids of each language
    langCentroids = []
    for language, df in data_by_lang:
        df.pop("Language")
        df.loc[language] = df.mean()
        df = df.iloc[-1]
        
        langCentroids.append(df)
        
    
    i = 0
    
   
    while i < len(d2v.index):
        
        if pd.isna(d2v.at[i, "Language"]):
            
            currRow = d2v.iloc[i].iloc[:-1]
            
            bestLang = langClass(langCentroids, currRow)
            d2v.at[i, "Language"] = bestLang
        i += 1
    
    
    
    bestLang = langClass(langCentroids, currRow)
    #print(currRow)
   # print(langCentroids)
    print(bestLang)
    print(d2v)
   
    return d2v

doc2vecLangs = languageClassifier("Data/book_rating_train.csv", "Data/book_text_features_doc2vec/train_desc_doc2vec100.csv")

In [20]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


'''Seems to be a good model, has good counts, verified a few instances seems to perform great in fact'''
def learnLanguage(file, doc2vec):
    
    
    data = pd.read_csv(file)
    X = pd.read_csv(doc2vec, index_col = False, delimiter = ',', header = None)
    y = data["Language"]
    
    y = pd.Series(y)
    # Assuming you have a feature matrix `X` and a target variable `y`
    # X should contain other features like doc2vec and word frequency counts
    # y should contain the language labels (with missing values)

    # Split the dataset into instances with and without missing language values
    X_with_language = X[~y.isnull()]
    y_with_language = y[~y.isnull()]
    X_missing_language = X[y.isnull()]

    # Split the dataset with language into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_with_language, y_with_language, test_size=0.2, random_state=42)

    # Train a Random Forest classifier on the instances with language
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)

    # Predict the missing language values using the trained model
    imputed_language = rf.predict(X_missing_language)

    # Merge the imputed language values with the original dataset
    y[y.isnull()] = imputed_language

    # Now you can proceed with your machine learning algorithm using the complete dataset
    return y

lol = learnLanguage("Data/book_rating_train.csv", "Data/book_text_features_doc2vec/train_desc_doc2vec100.csv")


In [29]:
print("Language counts with Random-Forest classifier")
print(lol.value_counts())


Language counts with Random-Forest classifier
eng    22555
spa      217
fre      175
ger       65
jpn       10
per        8
mul        7
por        5
lat        4
ita        4
zho        3
grc        2
heb        2
rus        2
ara        1
swe        1
frs        1
nld        1
Name: Language, dtype: int64


In [26]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(lol)

0        eng
1        eng
2        eng
3        eng
4        eng
5        eng
6        eng
7        eng
8        eng
9        eng
10       eng
11       eng
12       eng
13       eng
14       eng
15       eng
16       eng
17       eng
18       eng
19       eng
20       eng
21       eng
22       eng
23       eng
24       eng
25       eng
26       eng
27       eng
28       eng
29       eng
30       eng
31       eng
32       eng
33       eng
34       eng
35       eng
36       eng
37       eng
38       eng
39       eng
40       eng
41       eng
42       eng
43       eng
44       eng
45       eng
46       eng
47       eng
48       eng
49       eng
50       spa
51       eng
52       eng
53       eng
54       eng
55       eng
56       eng
57       eng
58       eng
59       eng
60       eng
61       eng
62       eng
63       eng
64       eng
65       eng
66       eng
67       eng
68       eng
69       eng
70       fre
71       eng
72       eng
73       eng
74       eng
75       eng
76       eng